<center>
<img src="../../img/ml_theme.png">
# Дополнительное профессиональное <br> образование НИУ ВШЭ
#### Программа "Практический анализ данных и машинное обучение"
<img src="../../img/faculty_logo.jpg" height="240" width="240">
## Автор материала: старший преподаватель Факультета Компьютерных Наук НИУ ВШЭ Кашницкий Юрий
</center>
Материал распространяется на условиях лицензии <a href="https://opensource.org/licenses/MS-RL">Ms-RL</a>. Можно использовать в любых целях, кроме коммерческих, но с обязательным упоминанием автора материала.

# <center>Занятие 8. Продвинутые методы классификации и регрессии. Xgboost</center>
## <center>Практика. Настройка Xgboost в задаче классификации</center>

In [ ]:
import numpy as np
import pandas as pd
%pylab inline
import seaborn as sns
from sklearn.model_selection import train_test_split, StratifiedKFold

**Считаем данные по оттоку клиентов телеком-оператора.**

In [ ]:
df = pd.read_csv('../../data/telecom_churn.csv')

In [ ]:
df.head()

**Сделаем минмимальную предобработку.**

In [ ]:
df['International plan'] = pd.factorize(df['International plan'])[0]

In [ ]:
df['Churn'] = df['Churn'].astype('int')

In [ ]:
df.info()

In [ ]:
df.drop(['State', 'Voice mail plan'], axis=1, inplace=True)

**Оставим 30% данных на финальную проверку моделей.**

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('Churn', axis=1), df['Churn'],
                                                    test_size=0.3, random_state=7)

**Зафиксируем объект StratifiedKFold, с его помощью будет проводить кросс-валидацию.**

In [ ]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=7)

**Используйте sklearn-обертку Xgboost для оценки ROC AUC на кросс-валидации с данными (X_train, y_train). Пока оставьте параметры XGBClassifier по умолчанию (только seed=7, это то же по сути, что random_state). Выведите средний ROC AUC на кросс-вадидации, используйте ранее созданный объект StratifiedKFold.**

In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score

In [ ]:
xgb_model1 = XGBClassifier(seed=7)

In [ ]:
%%time
np.mean(cross_val_score(xgb_model1, X_train, y_train, cv=skf,
                       scoring='roc_auc'))

**Обучите модель на (X_train, y_train) и выведите оценку важности признаков (plot_importance из xgboost).**

In [ ]:
''' ВАШ КОД ЗДЕСЬ '''

**Постройте кривые валидации по глубине деревьев от 2 до 7. Все прочие параметры кросс-валидации возьмите как на предыдущем шаге. Достаточно сохранять только средний AUC на валидационной части (на обучающей – не надо).**

In [ ]:
%%time
tree_depths = np.arange(2,8)
xgb_scores = []
for depth in tree_depths:
    ''' ВАШ КОД ЗДЕСЬ '''

In [ ]:
plot(tree_depths, xgb_scores, color='green', label='Xgb');
xlabel('Max depth')
ylabel('5-CV ROC AUC')
legend();

**Используйте RandomizedGridSearch, чтобы настроить параметры бустинга. Задайте 10 итераций (будут перебираться случайные 10 комбинаций параметров), а кросс-валидацию – как ранее. Выведите лучшие параметры и ROC AUC (поля best\_params\_ и best\_score\_ у обученного RandomizedGridSearch). В RandomizedGridSearch в качестве estimator можно предавать XGBClassifier с фиксированными параметрами, а в качестве параметров для перебора param_distributions – те, что мы проверяем. **

**В Python можно передать словарь аргументов и их значений вот так: SomeClass(\*\*params)**

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
xgb_params_dist_grid = {
    'max_depth': [2, 4, 6],
    'gamma': np.linspace(0, 0.5, 5),
    'subsample': [0.5, 0.75, 1.],
    'colsample_bytree': [0.5, 0.75, 1.]
}
xgb_params_fixed = {
    'objective': 'binary:logistic',
    'silent': 1,
    'seed': 7
}

In [ ]:
xgb_grid = RandomizedSearchCV ''' ВАШ КОД ЗДЕСЬ '''

In [ ]:
%%time
xgb_grid.fit ''' ВАШ КОД ЗДЕСЬ '''

In [ ]:
xgb_grid.best_params_

In [ ]:
xgb_grid.best_score_

**Теперь построим кривые валидации Xgboost с помощью функции cv из xgboost. Теперь построим кривые валидации Xgboost с помощью функции cv из xgboost. Важный момент: в общем случае лучшие параметры для XGBCLassifier могут не быть таковыми для оригинальной питоновской обертки Xgboost, это надо проверять. Либо пользоваться не оберткой sklearn, а, например hyperopt для настройки параметров.**

In [ ]:
from xgboost import cv, DMatrix

In [ ]:
dtrain = DMatrix(X_train, y_train)

Здесь указываем все параметры – и те что раньше были фиксированными, и те что менялись.

In [ ]:
best_xgb_params = cv ''' ВАШ КОД ЗДЕСЬ '''

**Сохраните результаты кросс-валидации в DataFrame cv_result, запустив 100 итераций бустинга. При необходимости посмотрите встроенную справку.**

In [ ]:
cv_result = cv(best_xgb_params, dtrain, num_boost_round=100, seed=7)

In [ ]:
cv_result.head()

In [ ]:
def plot_with_err(index, means, stds=None, **kwargs):
    lines = plt.plot(index, means, '-', **kwargs)
    plt.fill_between(index, means - stds, means + stds, edgecolor='none',
                     facecolor=lines[0].get_color(), alpha=0.2)

In [ ]:
plot_with_err(cv_result.index, cv_result['train-auc-mean'], cv_result['train-auc-std']);
plot_with_err(cv_result.index, cv_result['test-auc-mean'], cv_result['test-auc-std']);

**Как правило, бустинг лучще работает, если шаг градиентного спуска мал, а число итераций велико. Проще говоря, так он аккуратно находит минимум оптимизируемой функции.**

In [ ]:
best_xgb_params['eta'] = 0.003

In [ ]:
cv_result2 = cv ''' ВАШ КОД ЗДЕСЬ '''

In [ ]:
plot_with_err(cv_result2.index, cv_result2['train-auc-mean'], cv_result2['train-auc-std']);
plot_with_err(cv_result2.index, cv_result2['test-auc-mean'], cv_result2['test-auc-std']);

**Проверьте ROC AUC бустинга на оставленной выборке. Сравните начальную версию (без настройки параметров), промежуточную (с настроенными colsample_bytree, gamma, max_depth и subsample) и финальную (с настроенным вдобавок шагом и числом итераций).**

In [ ]:
## first model
roc_auc_score ''' ВАШ КОД ЗДЕСЬ '''

In [ ]:
## inermediate model
roc_auc_score ''' ВАШ КОД ЗДЕСЬ '''

In [ ]:
from xgboost import train

In [ ]:
## final model
final_xgb = train ''' ВАШ КОД ЗДЕСЬ '''

In [ ]:
roc_auc_score(y_test, final_xgb.predict(DMatrix(X_test)))